<a href="https://colab.research.google.com/github/22f3000982/DL_genai_project/blob/main/Model2_RoBERTa_Multilabel_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model-2 — Emotion Classification using RoBERTa (Pretrained Transformer)


In [2]:
!pip install transformers datasets wandb --quiet

import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from tqdm.auto import tqdm

import wandb
from transformers import AutoTokenizer, AutoModel
from datasets import Dataset

# W&B login (correct for Colab)
wandb.login()


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 22f3000982 (22f3000982-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
wandb.init(
    project="2025-sep-dl-genai-project",
    name="model2_roberta_multilabel",
    config={
        "model": "roberta-base",
        "epochs": 5,
        "batch_size": 16,
        "learning_rate": 2e-5,
        "max_length": 256,
    }
)
config = wandb.config


In [5]:
train_df = pd.read_csv("/content/train (3).csv")
labels = ["anger", "fear", "joy", "sadness", "surprise"]
train_df = train_df.dropna(subset=["text"]).reset_index(drop=True)
train_df[labels] = train_df[labels].astype(int)
train_df.head()


,id,text,anger,fear,joy,sadness,surprise,emotions
0,0,the dentist that did the work apparently did a...,1,0,0,1,0,['anger' 'sadness']
1,1,i'm gonna absolutely ~~suck~~ be terrible duri...,0,1,0,1,0,['fear' 'sadness']
2,2,"bridge: so leave me drowning calling houston, ...",0,1,0,1,0,['fear' 'sadness']
3,3,after that mess i went to see my now ex-girlfr...,1,1,0,1,0,['anger' 'fear' 'sadness']
4,4,"as he stumbled i ran off, afraid it might some...",0,1,0,0,0,['fear']


In [6]:
train_data, val_data = train_test_split(train_df, test_size=0.1, random_state=42)
train_data.shape, val_data.shape


((6144, 8), (683, 8))

In [7]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

def tokenize_batch(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=config.max_length
    )


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [8]:
from datasets import Dataset

train_ds = Dataset.from_pandas(train_data)
val_ds = Dataset.from_pandas(val_data)

train_ds = train_ds.map(tokenize_batch, batched=True)
val_ds = val_ds.map(tokenize_batch, batched=True)

cols = ["input_ids", "attention_mask"] + labels
train_ds.set_format("torch", columns=cols)
val_ds.set_format("torch", columns=cols)


Map:   0%|          | 0/6144 [00:00<?, ? examples/s]

Map:   0%|          | 0/683 [00:00<?, ? examples/s]

In [9]:
class RobertaMultiLabel(nn.Module):
    def __init__(self):
        super().__init__()
        self.roberta = AutoModel.from_pretrained("roberta-base")
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.roberta.config.hidden_size, 5)

    def forward(self, input_ids, attention_mask):
        out = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        cls = out.last_hidden_state[:, 0]
        cls = self.dropout(cls)
        return self.classifier(cls)


In [10]:
def collate(batch):
    return {
        "input_ids": torch.stack([x["input_ids"] for x in batch]),
        "attention_mask": torch.stack([x["attention_mask"] for x in batch]),
        "labels": torch.stack([torch.tensor([x[l] for l in labels]) for x in batch]).float()
    }

train_loader = DataLoader(train_ds, batch_size=config.batch_size, shuffle=True, collate_fn=collate)
val_loader = DataLoader(val_ds, batch_size=config.batch_size, shuffle=False, collate_fn=collate)


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RobertaMultiLabel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)
loss_fn = nn.BCEWithLogitsLoss()


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
best_f1 = 0

for epoch in range(config.epochs):
    model.train()
    for batch in tqdm(train_loader):
        ids = batch["input_ids"].to(device)
        mask = batch["attention_mask"].to(device)
        y = batch["labels"].to(device)
        logits = model(ids, mask)
        loss = loss_fn(logits, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # validation
    model.eval()
    preds, truths = [], []
    with torch.no_grad():
        for batch in val_loader:
            ids = batch["input_ids"].to(device)
            mask = batch["attention_mask"].to(device)
            y = batch["labels"].cpu().numpy()
            logits = model(ids, mask)
            p = torch.sigmoid(logits).cpu().numpy()
            preds.append(p)
            truths.append(y)

    preds = np.vstack(preds)
    truths = np.vstack(truths)
    pred_bin = (preds >= 0.5).astype(int)
    f1 = f1_score(truths, pred_bin, average="macro")
    acc = accuracy_score(truths, pred_bin)

    wandb.log({"epoch": epoch, "val_f1": f1, "val_acc": acc})
    print(f"Epoch {epoch} | F1: {f1:.4f} | Acc: {acc:.4f}")

    if f1 > best_f1:
        best_f1 = f1
        torch.save(model.state_dict(), "best_model_roberta.bin")
        print("Saved best model")


  0%|          | 0/384 [00:00<?, ?it/s]

Epoch 0 | F1: 0.7352 | Acc: 0.5271
Saved best model


  0%|          | 0/384 [00:00<?, ?it/s]

Epoch 1 | F1: 0.8121 | Acc: 0.6135
Saved best model


  0%|          | 0/384 [00:00<?, ?it/s]

Epoch 2 | F1: 0.8383 | Acc: 0.6589
Saved best model


  0%|          | 0/384 [00:00<?, ?it/s]

Epoch 3 | F1: 0.8564 | Acc: 0.6808
Saved best model


  0%|          | 0/384 [00:00<?, ?it/s]

Epoch 4 | F1: 0.8584 | Acc: 0.7262
Saved best model


In [13]:
wandb.finish()


epoch,▁▃▅▆█
val_acc,▁▄▆▆█
val_f1,▁▅▇██
epoch,4
val_acc,0.72621
val_f1,0.85839


In [14]:
import os

print("Files in current dir:")
print(os.listdir())

# agar direct nahi dikh raha to:
if os.path.exists("best_model_roberta.bin"):
    print("✅ best_model_roberta.bin FOUND")
else:
    print("❌ best_model_roberta.bin NOT FOUND")


Files in current dir:
['.config', 'best_model_roberta.bin', 'wandb', 'train (3).csv', 'sample_data']
✅ best_model_roberta.bin FOUND
